In [ ]:
import pandas as pd
import numpy as np
import math

from data_module_F import *
from model_module_F import *
from feature_module_F import *
from evaluation_module_F import *
from tensorflow.keras.models import model_from_json

In [ ]:
######## GPU RAM memory ##########
os.environ["CUDA_VISIBLE_DEVICES"]="0"

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 15*1GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=1024*5)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
            print(e)

In [3]:
path = 'dataset/' ## data path
exp = 'test_dl' ## experiment name 
seed = 1 ### the number of fold (from 1 to 5)
route = './models/'####Inset ## experiments folder 

epochs = 300
lr = 0.0005
kind = 'efficientNet' ## type of model architecture other options can be None: simple CNN or 'resnet'

dir_n = route + '{}_{}/'.format(exp,seed) ## Specific experiment dir to be created
create_path(dir_n)

tf.random.set_seed(155) ### seed for the experiement

In [4]:
db_train, X_train, y_train = read_db(path + 'fillCV_{}.csv'.format(seed),sp=True)
db_test, X_test, y_test = read_db(path + 'testCV_{}.csv'.format(seed),sp=True)

fill = db_train.copy()

samp_w_tr_all = pd.read_csv(path + 'samp_w_tr_{}.csv'.format(seed)).drop(['Unnamed: 0'],axis=1).loc[:,'0']

In [13]:
test = pd.DataFrame()

path_g = dir_n + 'all_models/'
create_path(path_g)

gap_fil = fill.copy()

for i in range(len(Traits[:2])):

    train_x, train_y = data_prep('400', gap_fil, Traits, i=i)
    samp_w_tr = samp_w_tr_all.loc[train_x.index]
    
        ##############
    scaler_list = save_scaler(train_y, save=True, dir_n = dir_n, k = i+1)

    val_x = train_x.sample(frac = 0.1,random_state = seed)
    val_y = train_y.loc[val_x.index,:]
    samp_w_val = pd.DataFrame(samp_w_tr).sample(frac = 0.1,random_state = seed)

    if(samp_w_tr is not None):
        if (samp_w_tr.sum().sum() !=0):    
            train_ds = dataset(train_x.drop(val_x.index), train_y.drop(val_y.index), pd.DataFrame(samp_w_tr).drop(samp_w_val.index), scaler_list, Traits, shuffle=True,augment=True)
            test_ds = dataset(val_x, val_y, samp_w_val, scaler_list, Traits)
    else:
        train_ds = dataset(train_x.drop(val_x.index), train_y.drop(val_y.index), None, scaler_list, Traits, shuffle=True,augment=True)
        test_ds = dataset(val_x, val_y, None, scaler_list, Traits)

        ##### Model definition  and taraining #######
    input_shape = train_x.shape[1]
    output_shape = train_y.shape[1]

    best_model = model_definition(input_shape, output_shape,lr= lr, kind=kind)

    # callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,restore_best_weights=True)

    checkpoint_path = path_g + 'checkpoint_{}'.format(i)
    create_path(checkpoint_path)

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_path + "/epoch{epoch:02d}-val_root_mean_squared_error{val_root_mean_squared_error:.2f}.hdf5",
    save_weights_only=True,
    monitor = 'val_root_mean_squared_error',
    mode='min',
    save_best_only=True)

    his = best_model.fit(train_ds,
                    validation_data = test_ds,
                    epochs = epochs,
                    verbose=1, callbacks = [model_checkpoint_callback])

    val_acc_per_epoch = his.history['val_root_mean_squared_error']
    best_epoch = val_acc_per_epoch.index(min(val_acc_per_epoch)) + 1    

    path_trial = path_g + "Model{}.json".format(i)
    path_best = checkpoint_path + "/epoch{0:02d}-val_root_mean_squared_error{1:.2f}.hdf5".format(best_epoch,min(val_acc_per_epoch))
    path_w = path_g + 'Trial{}_weights.h5'.format(i)
    save_model(best_model, path_trial, path_best, path_w)
    
    ######## Evaluation ##########
    test_x, test_y = data_prep('400', db_test, Traits, i=i)

    pred = scaler_list.inverse_transform(best_model.predict(test_x))
    pred_df = pd.DataFrame(pred, columns = test_y.columns+ ' Predictions')
    # pred_df.to_csv(path_g + 'Predictions_{}.csv'.format(i))

    obs_pf = pd.DataFrame(test_y)
    # obs_pf.to_csv(path_g + 'observations_{}.csv'.format(i))

    test = pd.concat([test,all_scores(Traits[i:i+1],Traits,obs_pf, pred_df,None)],axis=1)

# test.to_csv(path_g + 'Global_all.csv')

Epoch 1/2
446/446 [==============================] - 16s 32ms/step - loss: 53.1506 - root_mean_squared_error: 3.7392 - r_square: -12.9334 - val_loss: 25.8609 - val_root_mean_squared_error: 0.7764 - val_r_square: 0.3776
Epoch 2/2
446/446 [==============================] - 14s 31ms/step - loss: 12.8271 - root_mean_squared_error: 0.5188 - r_square: 0.7318 - val_loss: 12.8275 - val_root_mean_squared_error: 0.5295 - val_r_square: 0.7105
Epoch 1/2
348/348 [==============================] - 12s 33ms/step - loss: 90.4471 - root_mean_squared_error: 5.5747 - r_square: -30.0723 - val_loss: 37.2418 - val_root_mean_squared_error: 0.9410 - val_r_square: 0.1082
Epoch 2/2
348/348 [==============================] - 11s 32ms/step - loss: 25.2223 - root_mean_squared_error: 0.7593 - r_square: 0.4236 - val_loss: 25.0655 - val_root_mean_squared_error: 0.7537 - val_r_square: 0.4278


In [14]:
test

,LMA (g/m²),N content (mg/cm²)
r2_score,0.671747,0.316057
RMSE,38.879321,0.076818
nRMSE (%),10.778344,16.026837
MAE,23.207871,0.052986
Bias,2.787777,-0.009210
